# E-ID tutorial
This Jupyter notebook explores the basic components of E-ID. It focuses, in particular, on the cryptographic scheme BBS+ and how it's used to ensure the privacy requirements of E-ID.
These requirements include:
1. Selective Disclosure
2. Unlinkability
3. Zero-Knowledge Proofs

To understand these concepts in more detail, please refer to our blog post on [E-ID infrastructure](https://c4dt.epfl.ch/article/the-swiss-confederation-e-id-public-sandbox-trust-infrastructure-part-2/)
## Requirements
1. Typescript
2. [tslab](https://github.com/yunabe/tslab). This is the Typescript kernel for Jupyter lab.
3. [dock.io Typescript crypto lib](https://github.com/docknetwork/crypto-wasm-ts)

In [1]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

In [59]:
const JOHN_SNOW_BANK_IDENTITY = {
  fname: 'John',
  lname: 'Snow',
  highly_secretive: {
    secret: 'super-duper-secret',
    email: 'john.smith@example.com',
    SSN: '123-456789-0',
    'user-id': 'user:USR-22201'
  },
  location: {
    country: 'CH',
    city: 'Lausanne',
    address: 'C4DT, station 14'
  },
  timeOfBirth: 1662010849619,
  physical: {
    height: 170,
    weight: 78,
    race: 'dark elf'
  },
  score: 200
};

const SCHEMA = {
  fname: null,
  lname: null,
  highly_secretive: {
    secret: null,
    email: null,
    SSN: null,
    'user-id': null
  },
  location: {
    country: null,
    city: null,
    address: null
  },
  timeOfBirth: null,
  physical: {
    height: null,
    weight: null,
    race: null
  },
  score: null
};

In [27]:
const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

In [28]:
// Generating a keypair once signature parameters are created.
import { BBSPlusKeypairG2, BBSPlusSignatureParamsG1 } from '@docknetwork/crypto-wasm-ts';

// Deterministically generated params
const label = stringToBytes("Awesome key pair");
const cryptoParams = BBSPlusSignatureParamsG1.generate(14, label);

const keypair = BBSPlusKeypairG2.generate(cryptoParams);
const sk = keypair.secretKey;
const pk = keypair.publicKey;


In [29]:
import { Encoder } from '@docknetwork/crypto-wasm-ts'
import { BBSPlusSignatureG1 } from '@docknetwork/crypto-wasm-ts'

// The encoder has to be known and agreed upon by all system participants, i.e. signer, prover and verifier.
const defaultEncoder = (v: unknown) => {
  // @ts-ignore
  return BBSPlusSignatureG1.encodeMessageForSigning(stringToBytes(v.toString()));
};

const encoder = new Encoder(undefined, defaultEncoder);

In [53]:
import { BBSPlusSignatureG1 } from '@docknetwork/crypto-wasm-ts'


// The signing function will encode bytes to a field element as true is passed
const sig = BBSPlusSignatureG1.signMessageObject(JOHN_SNOW_BANK_IDENTITY, sk, label, encoder);

// As the messages are not encoded, pass true to the verification function to make it encode messages before verifying the signature.
sig.signature.verifyMessageObject(JOHN_SNOW_BANK_IDENTITY, pk, label, encoder);

const sigParams = getAdaptedSignatureParamsForMessages(cryptoParams, SCHEMA);

# Selective Disclosure 

In [31]:
// Both prover and verifier can independently create this struct
import { getAdaptedSignatureParamsForMessages, getRevealedAndUnrevealed } from '@docknetwork/crypto-wasm-ts'
const revealedNames = new Set(['fname', 'physical.height', 'location.city'])

// Prover prepares messages it wishes to reveal and hide.
const [revealedMsgs, unrevealedMsgs, revealedMsgsDecoded] = getRevealedAndUnrevealed(
    JOHN_SNOW_BANK_IDENTITY,
    revealedNames,
    encoder
);

In [32]:
revealedMsgsDecoded

{
  fname: 'John',
  physical: { height: 170 },
  location: { city: 'Lausanne' }
}


In [51]:
import {Statement, Statements, MetaStatements, ProofSpec, Witness, Witnesses, CompositeProof} from '@docknetwork/crypto-wasm-ts';


// Create a BBS statement - the information we reveal
const statement1 = Statement.bbsPlusSignatureProverConstantTime(sigParams, revealedMsgs, false);
const statements = new Statements();
statements.add(statement1);

const proofSpec = new ProofSpec(statements, new MetaStatements(), [], stringToBytes("Awesome Proof - test 001"));

const witness1 = Witness.bbsPlusSignatureConstantTime(sig.signature, unrevealedMsgs, false);
const witnesses = new Witnesses();
witnesses.add(witness1);

const nonce = stringToBytes('a unique nonce given by verifier');
const proof = CompositeProof.generate(proofSpec, witnesses, nonce);
proof

CompositeProof {
  value: Uint8Array(747) [
      1,   0,   0,   0,   0,   0,   0,   0,   0, 129,  85,  14,
    115, 236,  27, 121,  44,   9,  73,  93, 239, 213, 128, 196,
    190, 159, 128,  98,  37,  28, 176,  70, 124, 175, 102, 115,
      1, 139, 169, 190,  16, 248, 156,  54,  92,  96, 101,  37,
    170, 135,  94, 242,  11, 166, 166, 182, 248, 168,  84,  66,
     33, 123, 205,  74,  17, 158, 250, 163,  57, 254,  96,  55,
    177, 252,  92, 213, 143,   7,  81, 142, 159, 158, 146, 204,
     44, 161,  31,  66, 151, 182, 160, 101,  30,  97, 243, 149,
    150,  12, 173,  69,
    ... 647 more items
  ]
}


In [52]:
const verifierStatement1 = Statement.bbsPlusSignatureVerifierConstantTime(cryptoParams, pk, revealedMsgs, false);
const verifierStatements = new Statements();
verifierStatements.add(verifierStatement1);

const verifierProofSpec = new ProofSpec(verifierStatements, new MetaStatements(), [], stringToBytes("Awesome Proof - test 001"));

const verifierNonce = stringToBytes('A unique nonce given by verifier'); 

proof.verify(verifierProofSpec, verifierNonce)

{ verified: true, error: undefined }


# Zero-Knowledge proofs!
Here's we will provide a proof of the following: 
1. User's firstName, lastName, location
2. Score > 5

In [45]:
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();

const proverSetupParams: SetupParam[] = [];
proverSetupParams.push(SetupParam.legosnarkProvingKeyUncompressed(snarkProvingKey));

1


In [69]:
import { WitnessEqualityMetaStatement, getIndicesForMsgNames, MetaStatement, QuasiProofSpec } from '@docknetwork/crypto-wasm-ts';


// Prover prepares messages it wishes to reveal and hide.
const revealedParametersB = new Set(['fname', 'lname', 'location.city'])

const [revealedMsgsB, unrevealedMsgsB, revealedMsgsDecodedB] = getRevealedAndUnrevealed(
    JOHN_SNOW_BANK_IDENTITY,
    revealedParametersB,
    encoder
);

// Create a BBS signature
const statementB1 = Statement.bbsPlusSignatureProverConstantTime(sigParams, revealedMsgsB, false);
const statementB2 = Statement.boundCheckLegoProverFromSetupParamRefs(3, 10000, 0);
const statementsB = new Statements();
statementsB.add(statementB1);
statementsB.add(statementB2);

const scoreFieldIndex = getIndicesForMsgNames(['score'], SCHEMA)[0]  // 12 -- test if it stays the same always! -- should be!
const witnessEq = new WitnessEqualityMetaStatement();
witnessEq.addWitnessRef(0, scoreFieldIndex);
witnessEq.addWitnessRef(1, 0);
const metaStatementsB = new MetaStatements();
metaStatementsB.addWitnessEquality(witnessEq);

const proofSpecB = new QuasiProofSpec(statementsB, metaStatementsB, proverSetupParams);

const witnessB1 = Witness.bbsPlusSignatureConstantTime(sig.signature, unrevealedMsgsB, false);
const witnessB2 = Witness.boundCheckLegoGroth16(sig.encodedMessages['score']);
const witnessesB = new Witnesses();
witnessesB.add(witnessB1);
witnessesB.add(witnessB2);

const proof = CompositeProof.generateUsingQuasiProofSpec(proofSpecB, witnessesB);
proof
// Error in code:
// https://github.com/docknetwork/crypto/blob/a34298aeccd54105bab360d535ee6655f705e7ed/legogroth16/src/prover.rs#L573

Generating proof returned error: LegoGroth16Error(SynthesisError(Unsatisfiable))


### Notes:
**Make sure to use TLS to send the information to the verifier. Data is not encrypted on its own**